In [64]:
import os
from os.path import join as oj
import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
sys.path.insert(1, oj(sys.path[0], '../vision_fit'))  # insert parent path
import time
import numpy as np
from tqdm import tqdm
from copy import deepcopy
import pickle as pkl
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch
from torch.autograd import Variable
import torchvision.models as models
import warnings
warnings.filterwarnings("ignore")
import data
from max_corr_cnns import get_model_pretrained, lays_and_names

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# look at stats

In [ ]:
fnames = os.listdir('./max_corrs')
model_names = ['densenet', 'alexnet', 'resnet18', 'vgg16', 'inception_v3']
print('Model & Layer & Dim (RxC) & $||W_F||$ & Stable Rank & Mem. Coef. \\\\')
for model_name in model_names:
    fnames_m = [fname for fname in fnames if model_name in fname]
    if len(fnames_m) == 1:
        max_corrs = pkl.load(open(oj('max_corrs', fnames_m[0]), "rb"))
        lays, lay_names = lays_and_names(model_name)
        for lay, lay_name in zip(lays, lay_names):
            max_corr = max_corrs[lay_name]
            w = lay.weight            
            max_corr_norm = max_corr / w.norm(dim=1)
            svs = w.svd(compute_uv=False)[1].detach().cpu().numpy()**2
            print(model_name, '&', lay_name, '&', w.shape[0], 'x', w.shape[1],
                  '&', round(w.norm().item(), 2), '&', np.sum(svs) / np.max(svs),
                  '&', round(max_corr_norm.mean().item(), 3), '\\\\')
    else:
        pass
#         print(model_name, len(fnames_m), 'files found')

# load in imagenet dset

In [62]:
class p: pass
p.batch_size = 100
p.dset = 'imagenet'
train_loader, val_loader = data.get_data_loaders(p)

loading imagenet train dset...
done loading train dset 14.289691000000033 sec


In [67]:
# calculate max corrs for a linear layer
def linear_hook(module, act_in, act_out):
    # b is batch_size
    # input is (b x in_size)
    # weight is (out_size x in_size)
    # output is (out_1, ...., out_b)
    print(module.name)
    act_in_norm = act_in[0].t() / torch.norm(act_in[0], dim=1) # normalize each of b rows
    act_in_norm = act_in_norm.t() # transpose back to b x in_size
    
    Y = torch.matmul(act_in_norm, module.weight.t()) # Y is (b x out_size)
    
    corrs = torch.max(Y, dim=0)[0] # b (1-d)
    
    if not module.name in max_corrs:
        max_corrs[module.name] = corrs
    else:
        max_corrs[module.name] = torch.max(corrs, max_corrs[module.name]) # element wise max
    
model_name = 'vgg11' # alexnet, vgg16, inception_v3, resnet18, densenet
model = get_model_pretrained(model_name)
lays, names = lays_and_names(model_name)
for i, lay in enumerate(lays):
    lay.name = names[i]
    lay.register_forward_hook(linear_hook)
                              
max_corrs = {}   

# run - training set is about 14 mil
for i, x in enumerate(val_loader):
    ims = x[0].cuda()
    _ = model(ims)
    break